In [1]:
mitos = ["Menggunakan kacamata atau lensa kontak terlalu sering dapat membuat mata malas. Faktanya, kacamata atau lensa kontak membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.",
"Duduk terlalu dekat dengan layar komputer dapat merusak penglihatan. Meskipun dapat menyebabkan ketegangan mata, posisi duduk dekat dengan layar tidak merusak mata secara permanen.",
"Menggunakan wortel secara berlebihan akan meningkatkan penglihatan dengan cepat. Sementara wortel mengandung vitamin A yang baik untuk mata, konsumsi berlebihan tidak meningkatkan penglihatan.",
"Bersepeda dapat menyebabkan kebutaan. Ini adalah mitos; bersepeda sebenarnya adalah kegiatan yang baik untuk kesehatan secara keseluruhan, termasuk kesehatan mata.",
"Mata hanya perlu diperiksa jika ada masalah penglihatan. Tes mata secara berkala penting untuk mendeteksi masalah mata sejak dini, bahkan jika Anda tidak mengalami gejala.",
"Menggunakan tetes mata secara berlebihan dapat menyebabkan ketergantungan. Menggunakan tetes mata sesuai petunjuk dokter tidak menyebabkan ketergantungan.",
"Membaca dalam cahaya redup akan merusak mata. Ini hanya menyebabkan ketegangan mata sementara, tetapi tidak merusak mata secara permanen.",
"Menggosok mata saat gatal dapat menyebabkan penyebaran infeksi. Menggosok mata yang gatal dapat menyebabkan infeksi jika tangan Anda kotor, tetapi bukan karena tindakan menggosoknya itu sendiri.",
"Mata harus beristirahat sebentar setelah melihat layar komputer. Meskipun istirahat adalah ide yang baik, tidak ada bukti bahwa istirahat sebentar setelah melihat layar akan memperbaiki masalah penglihatan.",
"Terlalu banyak membaca dalam cahaya rendah atau membaca dengan buruk dapat merusak mata. Meskipun membaca dalam kondisi pencahayaan yang buruk dapat menyebabkan ketegangan mata sementara, itu tidak akan merusak mata secara permanen.",
]

fakta = ["Paparan sinar ultraviolet (UV) dari sinar matahari dapat menyebabkan kerusakan mata. Penggunaan kacamata hitam yang memiliki perlindungan UV dapat membantu mengurangi risiko ini.",
"Diabetes dapat mempengaruhi kesehatan mata. Kondisi ini dapat menyebabkan retinopati diabetik, yaitu kerusakan pembuluh darah pada retina yang dapat mengganggu penglihatan.",
"Perubahan penglihatan terjadi secara alami seiring bertambahnya usia. Ini termasuk presbyopia, yang menyebabkan kesulitan melihat objek dekat pada usia lanjut.",
"Merokok dapat meningkatkan risiko terjadinya katarak dan degenerasi makula, dua kondisi mata yang dapat menyebabkan kehilangan penglihatan.",
"Konsumsi makanan bergizi, seperti makanan kaya vitamin A, C, E, dan mineral seperti seng, dapat membantu menjaga kesehatan mata.",
"Mata kering adalah kondisi yang umum terjadi, terutama pada mereka yang banyak bekerja di depan layar komputer. Istirahat reguler dan tetes mata bisa membantu mengurangi ketidaknyamanan mata kering.",
"Penggunaan komputer dan perangkat elektronik lainnya dalam jangka waktu yang lama dapat menyebabkan Computer Vision Syndrome (CVS), menyebabkan gejala seperti mata kering, sakit kepala, dan ketegangan leher.",
"Bentuk fisik mata setiap orang berbeda. Ada tiga tipe mata utama: mata datar, mata cekung, dan mata normal. Ini dapat mempengaruhi pemilihan kacamata atau lensa kontak yang tepat.",
"Kacamata hitam yang baik melindungi mata dari sinar UV dan meminimalkan paparan sinar biru dari layar komputer atau perangkat elektronik lainnya.",
"Deteksi dini adalah kunci untuk mencegah banyak masalah mata yang serius. Berkunjung ke dokter mata secara berkala dapat membantu mendeteksi dan mengatasi masalah mata sejak dini."
]

In [2]:
import joblib
import tensorflow as tf
import numpy as np
import random
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
# Kasih target setiap kalimat, 0 = mitos, 1 = fakta
labelledMitos = [(kalimat, 0) for kalimat in mitos]
labelledFakta = [(kalimat, 1) for kalimat in fakta]
print(labelledMitos[0])
print(labelledFakta[0])

('Menggunakan kacamata atau lensa kontak terlalu sering dapat membuat mata malas. Faktanya, kacamata atau lensa kontak membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.', 0)
('Paparan sinar ultraviolet (UV) dari sinar matahari dapat menyebabkan kerusakan mata. Penggunaan kacamata hitam yang memiliki perlindungan UV dapat membantu mengurangi risiko ini.', 1)


In [5]:
# gabung fakta dan mitos, lalu shuffle
trainData = labelledMitos + labelledFakta
random.shuffle(trainData)
print(trainData[0])

('Mata kering adalah kondisi yang umum terjadi, terutama pada mereka yang banyak bekerja di depan layar komputer. Istirahat reguler dan tetes mata bisa membantu mengurangi ketidaknyamanan mata kering.', 1)


In [6]:
def clean_and_lower(text):
    cleaned_text = ''.join(char.lower() for char in text if char.isalnum() or char.isspace())
    return cleaned_text

In [7]:
# preprocess, hilangkan yang bukan alpha numerik dan lower
cleanData = [(clean_and_lower(kalimat), label) for kalimat, label in trainData]
random.shuffle(cleanData)
print(cleanData[0])

('mata kering adalah kondisi yang umum terjadi terutama pada mereka yang banyak bekerja di depan layar komputer istirahat reguler dan tetes mata bisa membantu mengurangi ketidaknyamanan mata kering', 1)


In [8]:
# Pisahkan antara attribut dan label
cleanAttr = [data for data, label in cleanData]
cleanLabel = [label for data, label in cleanData]
print(len(cleanAttr))
print(cleanLabel[0])

20
1


In [9]:
inputs = tokenizer(cleanAttr, padding=True, truncation=True, return_tensors="tf")
input_ids = inputs["input_ids"].numpy()
labels = np.array(cleanLabel)
outputs = bert_model(inputs)
SHAPE = inputs["input_ids"].shape[1]

In [10]:
input_layer = Input(shape=(inputs["input_ids"].shape[1],), dtype=tf.int32)
bert_embedding = bert_model(input_layer)["pooler_output"]
output_layer = Dense(1, activation='sigmoid')(bert_embedding)

In [11]:
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adamax(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(input_ids, labels, epochs=10)

Epoch 1/10
1/1 [==============================] - 48s 48s/step - loss: 0.7701 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 15s 15s/step - loss: 0.7786 - accuracy: 0.4500
Epoch 3/10
1/1 [==============================] - 14s 14s/step - loss: 0.6605 - accuracy: 0.6000
Epoch 4/10
1/1 [==============================] - 14s 14s/step - loss: 0.6304 - accuracy: 0.6500
Epoch 5/10
1/1 [==============================] - 14s 14s/step - loss: 0.6085 - accuracy: 0.7000
Epoch 6/10
1/1 [==============================] - 14s 14s/step - loss: 0.6015 - accuracy: 0.8500
Epoch 7/10
1/1 [==============================] - 14s 14s/step - loss: 0.5447 - accuracy: 0.9500
Epoch 8/10
1/1 [==============================] - 14s 14s/step - loss: 0.5459 - accuracy: 0.8500
Epoch 9/10
1/1 [==============================] - 14s 14s/step - loss: 0.4738 - accuracy: 0.7500
Epoch 10/10
1/1 [==============================] - 14s 14s/step - loss: 0.4779 - accuracy: 0.8500


In [31]:
kata = "Diabetes mempengaruhi kesehatan mata karena dapat menyebabkan retinopati diabetik yang dapat membuat susah melihat."
cleanKata = clean_and_lower(kata)
inputsCleanKata = tokenizer(cleanKata, padding=True, truncation=True, return_tensors="tf")
input_idsCleanKata = inputsCleanKata["input_ids"].numpy()

panjangTeks = input_idsCleanKata.shape[1]
panjangPred = SHAPE
butuh = np.zeros(panjangPred-panjangTeks, dtype=int)

manipulasi = np.append(input_idsCleanKata[0],butuh)
manipulasi = np.array([manipulasi])
print(SHAPE)
print(manipulasi)
hasil = model.predict(manipulasi)
print(hasil)

85
[[  101 14671  2033  8737 13159 15728  4048 17710  3366 12707  2319 22640
   8129  2050  4830  4502  2102  2273  6672  3676  2497  9126  2128 25690
  24952 22939 20915  5480  8675  4830  4502  2102  2033 14905  6692  2102
  10514  3736  2232 11463 19190  4017   102     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0]]
1/1 [==============================] - 6s 6s/step
[[0.81678206]]


In [32]:
# Save the model using joblib
joblib.dump(model, 'nlpModel.joblib')

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


['nlpModel.joblib']

In [39]:
model.save('bert_model_savedmodel')

TypeError: Binding inputs to tf.function `tf_bert_model_layer_call_and_return_conditional_losses` failed due to `too many positional arguments`.Received args: ({'input_ids': <tf.Tensor 'input_ids:0' shape=(None, None) dtype=int32>, 'attention_mask': <tf.Tensor 'attention_mask:0' shape=(None, None) dtype=int32>, 'token_type_ids': <tf.Tensor 'token_type_ids:0' shape=(None, None) dtype=int32>}, None, None, None, None, None, None, None, None, None, None, None, None, True, True) and kwargs: {} for signature: (input_ids=<object object at 0x000002380A794ED0>, attention_mask=<object object at 0x000002380A794ED0>, token_type_ids=<object object at 0x000002380A794ED0>, position_ids=<object object at 0x000002380A794ED0>, head_mask=<object object at 0x000002380A794ED0>, inputs_embeds=<object object at 0x000002380A794ED0>, encoder_hidden_states=<object object at 0x000002380A794ED0>, encoder_attention_mask=<object object at 0x000002380A794ED0>, past_key_values=<object object at 0x000002380A794ED0>, u